# Notebook for milestone presentation(data analysis)

In [ ]:
!{pip install pandas-profiling}

In [ ]:
import pandas as pd
from pandas_profiling import ProfileReport

df = pd.read_csv("out.csv")

In [ ]:
profile = ProfileReport(df, title="Pandas Profiling Report")
profile